In [1]:
from pydantic import Field
from lionagi.libs import func_call
from lionagi.core.prompt.prompt_template import ScoredTemplate
from lionagi.core.branch import Branch

In [2]:
class PredictTemplate(ScoredTemplate):
    template_name: str = "default_predict_template"
    sentence: str | list | dict = Field(
        default_factory=str, description="the given sentence(s) to predict"
    )
    num_sentences: int = Field(
        default_factory=int, description="the number of sentences to predict"
    )
    answer: str | list = Field(
        default_factory=str, description="the predicted sentence(s)"
    )
    signature: str = "sentence -> answer"

    def __init__(
        self,
        sentence=None,
        num_sentences=None,
        confidence_score=False,
        reason=False,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.sentence = sentence
        self.num_sentences = num_sentences
        self.task = f"predict the next {self.num_sentences} sentence(s)"

        if reason:
            self.output_fields.append("reason")

        if confidence_score:
            self.output_fields.append("confidence_score")


async def predict(
    sentence,
    num_sentences=1,
    confidence_score=False,
    reason=False,
    retries=2,
    delay=0.5,
    backoff_factor=2,
    default_value=None,
    timeout=None,
    branch_name=None,
    system=None,
    messages=None,
    service=None,
    sender=None,
    llmconfig=None,
    tools=None,
    datalogger=None,
    persist_path=None,
    tool_manager=None,
    **kwargs,
):

    branch = Branch(
        name=branch_name,
        system=system,
        messages=messages,
        service=service,
        sender=sender,
        llmconfig=llmconfig,
        tools=tools,
        datalogger=datalogger,
        persist_path=persist_path,
        tool_manager=tool_manager,
    )

    predict_template = PredictTemplate(
        sentence=sentence,
        num_sentences=num_sentences,
        confidence_score=confidence_score,
        reason=reason,
    )

    await func_call.rcall(
        branch.chat,
        prompt_template=predict_template,
        retries=retries,
        delay=delay,
        backoff_factor=backoff_factor,
        default=default_value,
        timeout=timeout,
        **kwargs,
    )

    return predict_template


In [3]:
sentence = "a blue whale chasing a big white shark"
out_ = await predict(sentence, reason=True, confidence_score=True)

In [4]:
print("Answer: ", out_.answer)
print("Reason: ", out_.reason)
print("Confidence: ", out_.confidence_score)

Answer:  The shark tries to escape quickly.
Reason:  Blue whales, being much larger, typically do not chase sharks as they feed on krill. However, in a hypothetical scenario where a blue whale is chasing a big white shark, it would likely be because the shark entered its territory, leading the shark to try to escape.
Confidence:  0.65
